In [1]:
%load_ext autoreload
%autoreload 2

from coeditor.common import *
import os

from coeditor.dataset import TokenizedEditDataset, is_repetitive_edit
from coeditor.model import *
from coeditor.encoding import TokenizedEdit, decode_tokens, WindowArgs, tokens_to_change
import shutil
import random

os.chdir(proj_root())

In [2]:
wargs = WindowArgs(4096)

# test_data_name = "medium"
test_data_name = "SPOT"
# data_dir = get_dataset_dir(test_data_name) / "tokenized-file_based" / "test.pkl"
data_dir = get_dataset_dir(test_data_name) / "tokenized-file_collapsed" / "test.pkl"
test_data: TokenizedEditDataset = pickle_load(data_dir)
test_data = test_data.map(lambda e: e.truncate_ctx(wargs))

rep_edits = pfilter(is_repetitive_edit, test_data.all_edits())
print("Total edits: ", len(list(test_data.all_edits())))
print("Repetitive edits: ", len(rep_edits))


filtering: 100%|██████████| 1311/1311 [00:00<00:00, 1484.06it/s]

Total edits:  1311
Repetitive edits:  88


In [3]:
# model_dir = get_model_dir(trained=True) / "coeditor-medium"
model_dir = get_model_dir(trained=True) / "coeditor-medium-collapse-shuffle"
model = CoeditorModel.load_pretrained(model_dir)
model.to("cuda:0")

eval_args = EvalArgs(4096 * 2, WindowArgs(4096))


In [4]:
rep_data = TokenizedEditDataset.from_edits(rep_edits)
rep_result = model.eval_loss_on_data(rep_data, eval_args)
display(rep_result)


evaluate loss: 100%|██████████| 34/34 [00:19<00:00,  1.75batch/s]


{'loss_per_ex': (mean=1.8138, weight=88),
 'loss_per_tk': (mean=0.02773, weight=5756),
 'prob_per_ex': (mean=0.55377, weight=88)}

In [14]:
sub_data = TokenizedEditDataset.from_edits(random.sample(test_data.all_edits(), 50))

In [15]:
dec_args = DecodingArgs(num_beams=1)
dec_result = model.predict_on_data(sub_data, eval_args, dec_args)
display(dec_result.exact_match_accuracy())


decoding: 100%|██████████| 19/19 [00:35<00:00,  1.85s/batch]


(mean=0.08, weight=50)

In [16]:
max_saved_samples = 200
random.seed(42)
exs_to_save = list(range(len(dec_result.predictions)))
random.shuffle(exs_to_save)
out_dir = Path("output/inspect_coeditor") / model_dir.name
dec_result.save_examples_to_dir(out_dir, exs_to_save[:max_saved_samples])

saving examples: 100%|██████████| 50/50 [00:00<00:00, 83.71it/s]
